# 梯度下降

到目前为止，我们已经构建了一个最简单的神经网络模型的基本框架：

* 模型参数：权重和偏置；
* 模型逻辑：前向传播和损失计算。

其中，模型逻辑是固定的，**网络模型的准确性取决于模型参数是否合理**。

那么，如何获得合理的参数？

答案是：让网络模型从数据中学习，这个过程称为**模型训练**（Model Training）。

## 模型训练

模型训练的目标很明确：**让损失函数的输出（损失值）尽可能小**。

神经元模型的本质是线性回归。我们可以对损失函数使用微积分的**梯度下降法**（Gradient Descent），来寻找更优的参数。

---

如果我们把推理函数代入损失函数，就会发现**损失函数是模型参数的函数**。可以表示为 $L(w, b)$。

在梯度下降法中，**梯度**（Gradient）是损失函数在某一点增加最快的方向，实质就是损失函数的导数。可以把梯度表示为：

$$
\nabla L = \left( \frac{\partial L}{\partial w}, \frac{\partial L}{\partial b} \right)
$$

**下降**（Descent）就是让模型参数沿着梯度相反的方向变化，从而达到让损失值变小的目的：

$$
w_{\text{new}} = w_{\text{old}} - \frac{\partial L}{\partial w}
$$
$$
b_{\text{new}} = b_{\text{old}} - \frac{\partial L}{\partial b}
$$

In [1]:
import numpy as np

## 数据

### 特征、标签

In [2]:
feature = np.array([28.1, 58.0])
label = np.array([165])

## 模型

### 参数：权重、偏置

In [3]:
weight = np.ones((1, 2)) / 2
bias = np.zeros(1)

### 推理函数

In [4]:
def forward(x, w, b):
    return x @ w.T + b

### 损失函数（均方误差）

In [5]:
def mse_loss(p, y):
    return np.mean(np.square(y - p))

### 梯度函数

损失值是预测值的函数（损失函数）；预测值是模型参数的函数（推理函数）。

根据微积分的**链式规则**（Chain Rule），我们可以分别计算两个函数的导数，然后将其相乘，从而获得损失值关于模型参数的导数。

---

损失函数的公式是：$L = (y - p)^2$，导数公式是：

$$
\frac{\partial L}{\partial p} = -2(y - p)
$$

推理函数的公式是：$p = w \cdot x + b$，权重和偏置的导数公式分别是：

$$
\frac{\partial p}{\partial w} = x
$$
$$
\frac{\partial p}{\partial b} = 1
$$

因此，根据链式规则，权重和偏置的梯度分别是：

$$
\frac{\partial L}{\partial w} = \frac{\partial L}{\partial p} \cdot \frac{\partial p}{\partial w} = -2(y - p) \cdot x
$$
$$
\frac{\partial L}{\partial b} = \frac{\partial L}{\partial p} \cdot \frac{\partial p}{\partial b} = -2(y - p)
$$

---

梯度函数将计算权重梯度和偏置梯度的共同部分：$-2(y - p)$。

在深度学习中，这个共同部分通常被称为**误差项**（Delta）。它是从损失函数反向传播的第一棒：

$$
\delta = -2(y - p)
$$

In [6]:
def gradient(p, y):
    return - 2 * (y - p)

### 反向函数

反向函数将根据梯度调整权重和偏置。这种数据从输出向输入方向的流动，被称为**反向传播**（Backpropagation）：

$$
w = w - \delta \cdot x
$$
$$
b = b - \delta \cdot 1
$$

In [7]:
def backward(x, d, w, b):
    w -= d * x
    b -= d
    return w, b

## 验证

### 推理

In [8]:
prediction = forward(feature, weight, bias)
print(f'prediction: {prediction}')

prediction: [43.05]


### 评估

In [9]:
loss = mse_loss(prediction, label)
print(f'loss: {loss}')

loss: 14871.802500000002


## 训练

现在，我们根据梯度下降的理论来进行第一次的模型训练。

### 梯度计算

In [10]:
delta = gradient(prediction, label)
print(f'delta: {delta}')

delta: [-243.9]


### 反向传播

In [11]:
weight, bias = backward(feature, delta, weight, bias)
print(f"weight: {weight}")
print(f"bias: {bias}")

weight: [[ 6854.09 14146.7 ]]
bias: [243.9]


经过一次模型训练，权重和偏置都有了明显的变化。效果如何呢？

### 重新评估

In [12]:
prediction = forward(feature, weight, bias)
loss = mse_loss(prediction, label)
print(f'loss: {loss}')

loss: 1026548766283.6302


不幸的是，损失值严重恶化。这又是为什么呢？

## 课后练习

思考一下，什么原因造成了损失值的恶化？有什么办法可以解决这个问题？